# Diagrama de Especiación en Python

En este notebook vamos a ver como podemos calcular la fracción molar de todas las especies generadas en la disociación de un ácido poliprótico a lo largo de la escala de pH

## CALCULOS PARA EL DIAGRAMA DE ESPECIACION
........................................

Estas funciones, dado un conjunto de pkas, calculan las fracciones
molares de cada especie para cada punto en la escala de pH. 

Se da como ejemplo particular la especiacion de un ácido diprótico
y luego se extiende a la fórmula general de un ácido poliprótico.

Para un ácido poliprótico definido genericamente como HNA, donde N es el
número de protones capaces de ser intercambiados, si se conocen
sus constantes de disociación puede calcularse la fracción molar
de cada una de las especies con distinto grado de disociación para cada pH.

Llamemos P al polinomio definido como

P = [H+]^N + K1*[H+]^(N-1) + K1*K2*[H+]^(N-2) + ... + K1*K2*...*KN*[H+]^(N-N = 0)

y Llamamos Tk a cada término del polinomio, de forma tal que

P = T0 + T1 + T2 + ... + TN

Vemos que el polinomio tiene N+1 términos, es decir, hay un término en el
polinomio por cada especie en el equilibrio. 

Si se trabaja con los balances de masa y las ecuaciones de equilibrio puede
llegarse a la siguiente expresión:

--------------------------------------
|                                     |
|   XHkA = TN-k / P ; k=0,1,2,...,N   |
|                                     |
---------------------------------------

In [7]:
import numpy as np
import pandas as pd

### Funciones

A continuación definimos las funciones que nos permitirán calcular la fracción molar de cada especie para un dado pH

In [49]:
def generateColNames(protons):
    colNames = []
    for i in range(0,protons+1):
        if(i == 0):
            colNames.append("A")
        elif(i == 1):
            colNames.append("HA")
        else:
            colNames.append("H"+ str(i) + "A")
    return colNames

def constantFromPka(pka):
    return 10**(-pka)

def poliproticSpeciation(pkas, res=10):
    # Ordeno pkas y obtengo ctes
    pkas = sorted(pkas)
    ks = list(map(constantFromPka, pkas))
    N = len(pkas)
    # Genero los nombres de las especies
    colNames = generateColNames(N)
    
    # Genero una lista de valores de pH entre 0 y 14
    pHvals = np.linspace(0, 14, res)
    data = {"pH": pHvals}
    for name in colNames:
        data[name] = []
    # Genero los coeficientes del polinomio
    # [1, k1, k1*k2,...]
    coefs = [1]
    for k in ks:
        coefs.append(coefs[-1]*k)
        
    # calculo las fracciones molares de cada especie:  
    for pH in pHvals:
        # Obtengo la conc de protones al pH dado
        H = 10**(-pH)
        # Calculo los términos del polinomio para ese pH
        pTerms = []
        for i, coef in enumerate(coefs):
            pTerms.append(coef*(H**(N-i)))
        # Uso la fórmula para calcular la fracción
        # molar de cada especie
        for j, name in enumerate(colNames):
            data[name].append(pTerms[N-j]/sum(pTerms))
    return data

### Cálculos
Reemplaza los siguientes valores con los del ácido de interés

In [53]:
pkas = [2,4,6]
res  =10

In [54]:
data = poliproticSpeciation(pkas, res)
df = pd.DataFrame(data = data)
df

,pH,A,HA,H2A,H3A
0,0.000000,9.900980e-13,9.900980e-07,9.900980e-03,9.900980e-01
1,1.555556,3.411245e-08,9.491992e-04,2.641203e-01,7.349305e-01
2,3.111111,1.382310e-04,1.070273e-01,8.286733e-01,6.416117e-02
3,4.666667,3.677020e-02,7.921899e-01,1.706722e-01,3.677020e-04
4,6.222222,6.237999e-01,3.739582e-01,2.241820e-03,1.343936e-07
5,7.777778,9.835900e-01,1.640727e-02,2.736898e-06,4.565420e-12
6,9.333333,9.995361e-01,4.639435e-04,2.153435e-09,9.995361e-17
7,10.888889,9.999871e-01,1.291533e-05,1.668079e-12,2.154407e-21
8,12.444444,9.999996e-01,3.593812e-07,1.291549e-15,4.641587e-26
9,14.000000,1.000000e+00,1.000000e-08,1.000000e-18,1.000000e-30


## Gráfico